# Dreem Project - CNN

In [1]:
import h5py
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## Data (NO NEED TO RE-RUN)

In [2]:
root_path = "original_data/"

y_train = pd.read_csv(root_path + "y_train.csv").as_matrix()[:, 1].squeeze()
X_train = h5py.File(root_path + "X_train.h5", "r")

#y_test = pd.read_csv("y_test.csv").as_matrix()[:, 1].squeeze()
X_test = h5py.File(root_path + "X_test.h5", "r") #we don't really care about that for now

/home/basile/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
def label_columns(df):
    l_columns = [
        'num_pso', 
        'mean_amp_pso',
        'mean_dur_pso',
        'amp_cso',
        'dur_cso',
        'curr_sleep_stage',
        'time_since_sleep',
        'time_in_ds',
        'time_in_ls',
        'time_in_rs',
        'time_in_ws'
    ]
    
    for i in range(12, 1261+1):
        l_columns.append('eeg_signal_%s'%(i-12+1))
    
    df.columns = l_columns

In [4]:
df_train = pd.DataFrame(data=X_train["features"][:])

# Label columns
label_columns(df_train)

# Add dummies
df_train = pd.get_dummies(
    df_train,
    prefix=['curr_sleep_stage'],
    columns=['curr_sleep_stage']
)

# Add outcome variable 'SO'
y_train_df = pd.DataFrame(
    y_train,
    columns=['SO'],
    index=df_train.index  #!important to keep same index
)
df_train['SO'] = y_train_df['SO']


df_train.head()

,num_pso,mean_amp_pso,mean_dur_pso,amp_cso,dur_cso,time_since_sleep,time_in_ds,time_in_ls,time_in_rs,time_in_ws,...,eeg_signal_1244,eeg_signal_1245,eeg_signal_1246,eeg_signal_1247,eeg_signal_1248,eeg_signal_1249,eeg_signal_1250,curr_sleep_stage_2.0,curr_sleep_stage_3.0,SO
0,237.0,152.658761,341.523207,128.017491,429.0,11379.0,2730.0,3780.0,0.0,480.0,...,7.305480,5.344436,2.674903,-0.055816,-1.212385,-2.461937,-4.930397,0,1,0
1,176.0,146.883435,338.039773,119.130849,196.0,6721.0,2580.0,2100.0,0.0,480.0,...,3.975107,1.000240,2.870631,7.071897,7.848365,4.033517,-2.110046,1,0,1
2,456.0,152.376541,335.629386,164.292580,417.0,26832.0,3240.0,7440.0,2130.0,750.0,...,5.708701,-0.753271,-5.627993,-9.804085,-12.863908,-11.951175,-5.531799,1,0,1
3,21.0,139.720772,336.285714,159.237082,407.0,1289.0,0.0,60.0,0.0,450.0,...,6.335396,-4.008689,-8.589818,-5.876062,0.166707,6.054539,12.086351,0,1,1
4,72.0,140.649432,349.875000,130.184278,297.0,2262.0,630.0,960.0,0.0,450.0,...,9.222596,21.236168,28.245889,27.024864,17.794644,7.602379,-4.548318,0,1,2


In [5]:
df_train.to_hdf('df_train_CNN.h5', key='df_train_CNN')

## Split data

In [2]:
df_train = pd.read_hdf('df_train_CNN.h5')

var_to_pred = 'SO'
X_train, X_val, y_train, y_val = train_test_split(df_train.loc[:, df_train.columns != var_to_pred],
                                                  df_train[var_to_pred],
                                                  test_size=0.10,
                                                  random_state=0,
                                                  stratify=df_train[var_to_pred])
y_train = pd.DataFrame(y_train)
y_val = pd.DataFrame(y_val)

In [3]:
from keras.utils import to_categorical

# Convert labels to categorical one-hot encoding
y_train = to_categorical(y_train.values, num_classes=3)
y_val = to_categorical(y_val.values, num_classes=3)

Using TensorFlow backend.


In [4]:
df_train.head()

,num_pso,mean_amp_pso,mean_dur_pso,amp_cso,dur_cso,time_since_sleep,time_in_ds,time_in_ls,time_in_rs,time_in_ws,...,eeg_signal_1244,eeg_signal_1245,eeg_signal_1246,eeg_signal_1247,eeg_signal_1248,eeg_signal_1249,eeg_signal_1250,curr_sleep_stage_2.0,curr_sleep_stage_3.0,SO
0,237.0,152.658761,341.523207,128.017491,429.0,11379.0,2730.0,3780.0,0.0,480.0,...,7.305480,5.344436,2.674903,-0.055816,-1.212385,-2.461937,-4.930397,0,1,0
1,176.0,146.883435,338.039773,119.130849,196.0,6721.0,2580.0,2100.0,0.0,480.0,...,3.975107,1.000240,2.870631,7.071897,7.848365,4.033517,-2.110046,1,0,1
2,456.0,152.376541,335.629386,164.292580,417.0,26832.0,3240.0,7440.0,2130.0,750.0,...,5.708701,-0.753271,-5.627993,-9.804085,-12.863908,-11.951175,-5.531799,1,0,1
3,21.0,139.720772,336.285714,159.237082,407.0,1289.0,0.0,60.0,0.0,450.0,...,6.335396,-4.008689,-8.589818,-5.876062,0.166707,6.054539,12.086351,0,1,1
4,72.0,140.649432,349.875000,130.184278,297.0,2262.0,630.0,960.0,0.0,450.0,...,9.222596,21.236168,28.245889,27.024864,17.794644,7.602379,-4.548318,0,1,2


In [5]:
eeg_indexes = np.arange(10, df_train.shape[1] - 3)

X_train_eeg = X_train.iloc[:, eeg_indexes]
X_val_eeg = X_val.iloc[:, eeg_indexes]

# we feed 2 downsampled datasets + the original dataset = 3
model_train = [
    X_train_eeg.iloc[:, np.arange(0, 1250, 5)].values[:,:,None], # small
    X_train_eeg.iloc[:, np.arange(0, 1250, 2)].values[:,:,None], # medium
    X_train_eeg.values[:,:,None],                                # original
#     X_train.iloc[:, 0:10],                                       # auxiliary
]

model_val = [
    X_val_eeg.iloc[:, np.arange(0, 1250, 5)].values[:,:,None], # small
    X_val_eeg.iloc[:, np.arange(0, 1250, 2)].values[:,:,None], # medium
    X_val_eeg.values[:,:,None],                                # original
#     X_val.iloc[:, 0:10],                                       # auxiliary
]

In [11]:
# Inspired from https://towardsdatascience.com/how-to-use-convolutional-neural-networks-for-time-series-classification-56b1b0a07a57

from keras.models import Model, Sequential
from keras.layers import Lambda, Dense, Dropout, Input, Concatenate
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.layers import LSTM
from keras.backend import expand_dims, squeeze

# Base model
# 3 of them are run in parallel with each a downsampled version of the data
def get_base_model(input_len, kernel_size):
    nb_filters = 128
    
    model = Sequential([
#         Conv1D(nb_filters, kernel_size, activation='relu', input_shape=(input_len, 1)),
#         Conv1D(nb_filters, kernel_size, activation='relu'),
#         MaxPooling1D(4),
#         Conv1D(nb_filters*2, kernel_size, activation='relu'),
#         Conv1D(nb_filters*2, kernel_size, activation='relu'),
#         GlobalMaxPooling1D(),
#         MaxPooling1D(8),
#         Lambda(lambda y: squeeze(y, axis=0)),
#         LSTM(10),
        Dense(5, activation='relu'),
#         Dropout(0.3),
    ])
    
    return model

# Main model
def main_model(inputs_lens = [250, 625, 1250, 10], kernel_sizes = [4, 8, 16]):
    # the inputs to the branches are the original time series, and its down-sampled versions
    input_smallseq = Input(shape=(inputs_lens[0], 1), name='eeg_small')
    input_medseq   = Input(shape=(inputs_lens[1], 1), name='eeg_medium')
    input_origseq  = Input(shape=(inputs_lens[2], 1), name='eeg_original')
    
    # auxiliary data
#     input_aux      = Input(shape=(inputs_lens[3],), name='aux')

    # the more down-sampled the time series, the shorter the corresponding filter
    base_net_small    = get_base_model(inputs_lens[0], kernel_sizes[0])
    base_net_med      = get_base_model(inputs_lens[1], kernel_sizes[1])
    base_net_original = get_base_model(inputs_lens[2], kernel_sizes[2])
    
    embedding_small    = base_net_small(input_smallseq)
    embedding_med      = base_net_med(input_medseq)
    embedding_original = base_net_original(input_origseq)
    
    # concatenate all the outputs
    x = Concatenate()([embedding_small, embedding_med, embedding_original])
#     x = Dropout(0.5)(x)
    
    # Conv layer
#     x = Lambda(lambda y: expand_dims(y, axis=-1))(x)
#     x = Conv1D(512, 4, activation='relu')(x)
#     x = GlobalMaxPooling1D()(x)
#     x = MaxPooling1D(10)(x)
#     x = Lambda(lambda y: squeeze(y, axis=0))(x)
#     x = LSTM(150)(x)
#     x = Dropout(0.3)(x)
    
    # auxiliary output
#     aux_out = Dense(3, activation='softmax')(x)
    
    # merge with aux data
#     merged_with_aux = Concatenate()([x, input_aux])
    
    # dense architecture
#     stacked = Dense(128, activation='relu')(merged_with_aux)
#     stacked = Dense(128, activation='relu')(stacked)
#     x = Dense(128, activation='relu')(x)
    
    # output layer
    out = Dense(3, activation='softmax')(x)
    
    model = Model(inputs=[input_smallseq, input_medseq, input_origseq], outputs=out)
#     model = Model(inputs=[input_smallseq, input_medseq, input_origseq, input_aux], outputs=[aux_out, out])
#     model = Model(inputs=[input_smallseq, input_medseq, input_origseq, input_aux], outputs=out)
    return model

In [12]:
BATCH_SIZE = 248
EPOCHS = 3

model = main_model()

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model.fit(model_train, [y_train, y_train],
#           batch_size=batch_size,
#           epochs=epochs,
#           validation_data=(model_val, [y_val, y_val]))

model.fit(model_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_data=(model_val, y_val))

Train on 235470 samples, validate on 26164 samples
Epoch 1/3
235470/235470 [==============================] - 438s 2ms/step - loss: 1.0498 - accuracy: 0.4518 - val_loss: 1.0374 - val_accuracy: 0.4658
Epoch 2/3
235470/235470 [==============================] - 495s 2ms/step - loss: 1.0128 - accuracy: 0.4842 - val_loss: 0.9975 - val_accuracy: 0.4999
Epoch 3/3
235470/235470 [==============================] - 467s 2ms/step - loss: 1.0258 - accuracy: 0.4737 - val_loss: 1.0355 - val_accuracy: 0.4689


- Batch size 128 & 8 epochs --> Train = 0.4358 & Test = 0.4359
- archi 2CNN-MaxPool-2CNN-GlobalMaxPool, Batch size 256 & 4 epochs --> Train = 0.5073 & Test = 0.5091
- 25 filters \+ stacked 10 features, Batch size 256 & 3 epochs --> Train = 0.4185 & Test = 0.4734
- 25 filters \+ stacked 10 features, Batch size 256 & 10 epochs --> Train = 0.4908 & Test = 0.5088
- \+ aux output, Batch size 256 & 10 epochs --> Train = 0.05019 & Test = 0.5045

In [14]:
model.predict(model_val)

array([[0.44115183, 0.3014133 , 0.25743487],
       [0.4569084 , 0.29650405, 0.2465876 ],
       [0.48024124, 0.29317138, 0.22658737],
       ...,
       [0.37078846, 0.3386435 , 0.2905681 ],
       [0.42868525, 0.30553266, 0.2657821 ],
       [0.28235856, 0.34383   , 0.37381145]], dtype=float32)

In [15]:
y_val

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [16]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
eeg_small (InputLayer)          (None, 250, 1)       0                                            
__________________________________________________________________________________________________
eeg_medium (InputLayer)         (None, 625, 1)       0                                            
__________________________________________________________________________________________________
eeg_original (InputLayer)       (None, 1250, 1)      0                                            
__________________________________________________________________________________________________
sequential_10 (Sequential)      (None, 5)            535         eeg_small[0][0]                  
____________________________________________________________________________________________

### Predictions

In [18]:
X_test_eeg = df_test.iloc[:, eeg_indexes]

model_test = [
    X_test_eeg.iloc[:, np.arange(0, 1250, 5)].values[:,:,None], # small
    X_test_eeg.iloc[:, np.arange(0, 1250, 2)].values[:,:,None], # medium
    X_test_eeg.values[:,:,None],                                # original
]

   eeg_signal_1  eeg_signal_2  eeg_signal_3  eeg_signal_4  eeg_signal_5  \
0   -115.857220   -113.989029   -112.478076   -110.306621   -109.061839   
1    -33.774921    -44.025140    -51.526478    -51.519891    -47.469593   
2    -47.189094    -47.192568    -48.777384    -53.342472    -57.437670   
3    -27.608822    -27.627443    -29.543476    -31.600733    -32.958107   
4    -30.656929    -21.554853    -10.721147     -1.645441      0.794205   

   eeg_signal_6  eeg_signal_7  eeg_signal_8  eeg_signal_9  eeg_signal_10  ...  \
0   -106.369929   -102.945979    -99.867133    -97.368306     -93.806783  ...   
1    -46.197608    -44.843232    -42.487383    -39.799901     -37.531789  ...   
2    -59.110127    -62.207202    -66.907604    -69.886341     -64.422704  ...   
3    -31.254558    -28.609017    -27.688751    -27.887819     -28.462239  ...   
4      1.492238      2.299947     -0.296050     -5.434099     -10.695539  ...   

   eeg_signal_1241  eeg_signal_1242  eeg_signal_1243  eeg_sign

In [1]:
df_test = pd.read_hdf('df_test_CNN.h5')

model.predict(df_test, batch_size=BATCH_SIZE)

NameError: name 'pd' is not defined